<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest
4. Consumer API keys: 
    
    Z22dLMPEOtCL2ayqnPTbFg3sK (API key)
    
    SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj (API secret key)
    
    Access token & access token secret

    14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb (Access token)

    nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313 (Access token secret)

    Read and write (Access level)




In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [4]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [5]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [0]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')


In [7]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 765
Rate limit reached. Sleeping for: 768


In [8]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print('%s %d %s %d' % ('Valid tweet count: ', len(f.readlines()), 'Error tweet count: ', len(error_tweets)))

Valid tweet count:  2339 Error tweet count:  17


In [0]:
# Read "tweet_json.txt" file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest

columns = ['tweet_id', 'text', 'retweet_count', 'favorite_count', 'retweeted', 'favorited', 'media_url']
tweets_mini_df = pd.DataFrame(columns = columns)

index = 0
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    for tweet in f:
        tweet_json = json.loads(tweet)
        tweet_id = tweet_json['id']
        text = tweet_json['text']
        retweet_count = tweet_json['retweet_count']
        favorite_count = tweet_json['favorite_count']
        retweeted = tweet_json['retweeted']
        favorited = tweet_json['favorited']
        try:
            media_url = tweet_json['entities']['media'][0]['media_url_https']
        except KeyError as e:
            media_url = ''

        tweets_mini_df.loc[index] = pd.Series({'tweet_id': tweet_id, 
                                        'text': text, 
                                        'retweet_count': retweet_count, 
                                        'favorite_count': favorite_count,
                                        'retweeted': retweeted,
                                        'favorited': favorited,
                                        'media_url': media_url})
        index +=1


### Assessing data

Key Points

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.

* Cleaning includes merging individual pieces of data according to the rules of tidy data.

* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.

* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


#### twitter-archive-enhanced.csv

In [10]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [11]:
tweets_df.query('name == "None"').sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
658,791672322847637504,NaN,NaN,2016-10-27 16:06:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When she says you're a good boy and you know y...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/791672322...,13,10,None,None,None,None,None
934,753420520834629632,NaN,NaN,2016-07-14 02:47:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we are witnessing an isolated squad of bo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/753420520...,11,10,None,None,None,None,None
1293,708026248782585858,NaN,NaN,2016-03-10 20:26:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Everybody stop what you're doing and watch thi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/708026248...,13,10,None,None,None,None,None
127,867900495410671616,NaN,NaN,2017-05-26 00:29:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Unbelievable. We only rate dogs. Please don't ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/867900495...,12,10,None,None,None,None,None
605,798576900688019456,NaN,NaN,2016-11-15 17:22:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Not familiar with this breed. N...,6.661041e+17,4.196984e+09,2015-11-16 04:02:55 +0000,https://twitter.com/dog_rates/status/666104133...,1,10,None,None,None,None,None


In [12]:
tweets_df.isnull().sum()
tweets_df.query('name == "None"').shape

(745, 17)

#### image_predictions.tsv

In [13]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [14]:
img_pred_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
610,680100725817409536,https://pbs.twimg.com/media/CW-loUBWYAAn2Cb.jpg,1,golden_retriever,0.698961,True,chow,0.145971,True,Pomeranian,0.034888,True
1713,818614493328580609,https://pbs.twimg.com/media/C1xNgraVIAA3EVb.jpg,4,Chihuahua,0.450722,True,Border_terrier,0.204177,True,beagle,0.092774,True
243,670452855871037440,https://pbs.twimg.com/media/CU3tUC4WEAAoZby.jpg,1,Arctic_fox,0.188174,False,indri,0.123584,False,malamute,0.080379,True
490,675531475945709568,https://pbs.twimg.com/media/CV_4ShmUYAA3wNu.jpg,1,Pembroke,0.918441,True,Cardigan,0.027339,True,Siberian_husky,0.020221,True
1918,855459453768019968,https://pbs.twimg.com/media/C98z1ZAXsAEIFFn.jpg,2,Blenheim_spaniel,0.389513,True,Pekinese,0.188220,True,Japanese_spaniel,0.082628,True


In [15]:
list(img_pred_df)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

#### tweets_mini_df

In [16]:
tweets_mini_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339 entries, 0 to 2338
Data columns (total 7 columns):
tweet_id          2339 non-null object
text              2339 non-null object
retweet_count     2339 non-null object
favorite_count    2339 non-null object
retweeted         2339 non-null object
favorited         2339 non-null object
media_url         2339 non-null object
dtypes: object(7)
memory usage: 146.2+ KB


In [17]:
tweets_mini_df.sample(5)
tweets_mini_df.query('tweet_id == 683357973142474752')

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url
1640,683357973142474752,"""Have a seat, son. There are some things we ne...",995,3052,False,False,https://pbs.twimg.com/media/CXvGbWeWMAcRbyJ.jpg


In [18]:
tweets_mini_df.isnull().sum()

tweet_id          0
text              0
retweet_count     0
favorite_count    0
retweeted         0
favorited         0
media_url         0
dtype: int64

#### Cleaning data

#### 1. twitter-archive-enhanced.csv
##### 1.1 Data Quality Issues:

1. There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. These need to be removed from the dataset as we only want original tweets (1.1.1)

2. Remove the following columns from the dataset. I don't think that we need these columns for our analysis (1.1.2)

    *  in_reply_to_status_id
    *  in_reply_to_user_id
    *  retweeted_status_id
    *  retweeted_status_user_id
    *  retweeted_status_timestamp
    *  expanded_urls

3. The "timestamp" column is represented as a string and needs to be converted to datetime (1.1.3)
4. Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements (1.1.4)
5. Dog stages are not fully recorded and many of them contain null values or values labeled as "None". We need to scan the tweet text and see if we can extract these values for each row (1.1.5)
6. *** Not Addressed *** There are 745 rows in this dataset where the name is marked as "None". I am not addressing this issue right now since I don't have sufficient data to obtain this information (1.1.6)

##### 1.2 Tidiness Issues:
1. Dog stages are represented in separate columns as a sparse-array matrix which is wasteful and also not scalable in case we identify new stages. These need to be converted to a single column titled "stage". If they are not mutually exclusive (meaning, if there are multiple tweets about the same dog at different points in time where their stages change), we can put them into a separate dataframe for normalization purposes (1.2.1)


#### 2. image-predictions.tsv

##### 2.1 Tidiness Issues:

1. In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence against each and a boolean flag indicating whether the image prediction is actually canine or not. That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. Addressing this tidiness issue will make it easier to fix the data quality issues identified below. We will begin with separating breed predictions to its own dataframe so that we can work on the data quality issues later (2.1.1)

##### 2.2 Data Quality Issues:

1. Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. We will then pick the prediction with the higher percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a single breed) and drop the ones with lower prediction confidence scores (2.2.1)
2. We can then merge this dataset back to the original dataset (2.2.2)
3. After merging, we will drop the following columns as they are no longer needed (2.2.3): 
    *  jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
    *  img_num (don't see how this information will be useful)
    *  p1
    *  p1_conf
    *  p1_dog
    *  p2
    *  p2_conf
    *  p2_dog
    *  p3
    *  p3_conf
    *  p3_dog




#### 3. tweet_mini_df

##### 3.1 Data Quality Issues
1. We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets (3.1.1)


#### 4. Tidiness Issues: combining data from all the datasets

##### 4.1 Tidiness

1. Having addressed the data quality and tidiness issues at the level of individual datasets, we can now merge the data in these datasets into one. Since the data granularity in each of these datasets is at the "tweet_id" level, we will use it to combine the data into a single dataset and persist it to a file (4.1.1)

##### 4.2 Data Quality

1. We will use "inner joins" to merge these datasets thus eliminating any rows for whcih no valid information is available online. For example, there are 17 tweet ids in the original "twitter-archive-enhanced" dataset for which the Twitter API returned errors and these will be eliminated during the merging process. (4.2.1)

In [19]:
# First, let us make clean copies of each of the dataframes and leave the originals intact

tweets_df_clean = tweets_df.copy()
img_pred_df_clean = img_pred_df.copy()
tweets_mini_df_clean = tweets_mini_df.copy()

print ('%s %d %s %d' %('Original Row Count: ', tweets_df.shape[0], 'Copy Row Count: ', tweets_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', img_pred_df.shape[0], 'Copy Row Count: ', img_pred_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', tweets_mini_df.shape[0], 'Copy Row Count: ', tweets_mini_df_clean.shape[0]))

Original Row Count:  2356 Copy Row Count:  2356
Original Row Count:  2075 Copy Row Count:  2075
Original Row Count:  2339 Copy Row Count:  2339


In [20]:
"""
1.1.1 Define
There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. 
These need to be removed from the dataset as we only want original tweets
"""

tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].shape[0]

181

In [0]:
"""
1.1.1 Code
"""

tweets_df_clean.drop(tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].index, inplace = True)


In [0]:
"""
1.1.1 Test
Assertion: Original DF Row Count - Cleaned DF Row Count = Retweet Count
"""

assert (tweets_df.shape[0] - tweets_df_clean.shape[0] == tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0])
# print('%s %d %s %d %s %d' % ('Org. Count: ', tweets_df.shape[0], 'New Count: ', tweets_df_clean.shape[0], 'Diff: ',  tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0]))

In [23]:
"""
1.1.2 Define
Remove the following columns from the dataset:

in_reply_to_status_id
in_reply_to_user_id
retweeted_status_id
retweeted_status_user_id
retweeted_status_timestamp
expanded_urls
"""
tweets_df_clean_drop_cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls']
tweets_df_clean[tweets_df_clean_drop_cols].head()

,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls
0,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...
1,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...
2,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...
3,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...
4,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...


In [0]:
"""
1.1.2 Code
"""
tweets_df_clean.drop(tweets_df_clean_drop_cols, axis = 1, inplace = True)

In [0]:
"""
1.1.2 Test
"""

assert not set(tweets_df_clean_drop_cols).issubset(tweets_df_clean.columns)

In [26]:
""" 
1.1.3 Define
The "timestamp" column is represented as a string and needs to be converted to datetime
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(8)
memory usage: 203.9+ KB


In [0]:
"""
1.1.3 Code
"""

tweets_df_clean['timestamp'] = pd.to_datetime(tweets_df_clean['timestamp'])

In [28]:
"""
1.1.3 Test
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 203.9+ KB


In [29]:
"""
1.1.4 Define
Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements
"""
tweets_df_clean.query('timestamp > "08/01/2017"')

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None,None


In [0]:
"""
1.1.4 Code
"""

tweets_df_clean.drop(tweets_df_clean.query('timestamp > "08/01/2017"').index, inplace = True)

In [0]:
"""
1.1.4 Test
"""

assert tweets_df_clean.query('timestamp > "08/01/2017"').shape[0] == 0

In [0]:
"""
1.1.5 Define
Dog stages are not fully recorded and many of them contain null values or values labeled as "None". 
We need to scan the tweet text and see if we can extract these values for each row
"""

dog_stages = ['doggo', 'floofer', 'pupper', 'puppo']
[tweets_df_clean[dog_stages] == 'None']

In [0]:
"""
1.1.5 Code
In this cleaning exercise, I am going to transpose the the "dog_stage columns" to rows first to make the next steps easier
"""

dog_stage_df = pd.melt(tweets_df_clean[['tweet_id', 'text', 'doggo', 'floofer', 'pupper', 'puppo']], id_vars = ['tweet_id', 'text'], var_name = 'dog_stage_col', value_name = 'dog_stage')

# Next, I am assigning the value "doggo" to the column 'dog_stage' in cases where the tweet text has the word "doggo" in it and the 'dog_stage' column has the value "None"
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('doggo ', case = False)].query('dog_stage_col == "doggo" and dog_stage == "None"').index, 'dog_stage'] = 'doggo'

# Next, I am repeating the above for other 'dog_stage' values
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('floofer ', case = False)].query('dog_stage_col == "floofer" and dog_stage == "None"').index, 'dog_stage'] = 'floofer'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('pupper ', case = False)].query('dog_stage_col == "pupper" and dog_stage == "None"').index, 'dog_stage'] = 'pupper'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('puppo ', case = False)].query('dog_stage_col == "puppo" and dog_stage == "None"').index, 'dog_stage'] = 'puppo'


In [87]:
"""
The test below confirm that there are multiple dog stages tied to a single tweet id.
Since I am not sure if this is good or bad data, I am going to err on the conservative side and treat it as good.
Therefore, i am going to retain this as a separate dataset until I find evidence to the contrary
However, I am going to get rid of all rows where the dog stage is None even after fixing the quality
"""
dog_stage_df[['tweet_id', 'dog_stage']].query('dog_stage != "None"').groupby('tweet_id').filter(lambda x: len(x) > 1).shape

(24, 2)

In [0]:
"""
We will begin with dropping the unnecessary columns
"""
# dog_stage_df.drop(['text', 'dog_stage_col'], axis = 1, inplace = True)
# dog_stage_df.info()

"""
Next, we will drop all rows where the dog_stage column contains the value "None"
"""

dog_stage_df.drop(dog_stage_df.query('dog_stage == "None"').index, inplace = True)


In [0]:
"""
1.1.5 Test by visual inspection
"""

assert dog_stage_df.query('dog_stage == "None"').shape[0] == 0